## Optimizing a Pipeline in Azure - Daniel Webb - 22nd September 2022

Note: Code from the Azure ML sample Jupyter Notebook below are used:  
```Samples\1.43.0\how-to-use-azureml\ml-frameworks\keras\train-hyperparameter-tune-deploy-with-keras\train-hyperparameter-tune-deploy-with-keras.ipynb```

## Set up workspace

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-208238
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-208238


## Set up compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "StandardD2CPUCluster"

# DW: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive run of the logistic regression model in train.py

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# DW: Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.001, 1), # Inverse of regularization strength
    "--max_iter": choice(50, 100, 150, 200) # Maximum number of iterations to converge
    })

# DW: Specify a Termination Policy (for Early Stopping)
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# DW: Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# DW: Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [4]:
# DW: Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38
Web View: https://ml.azure.com/runs/HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-208238/workspaces/quick-starts-ws-208238&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-09-21T21:22:17.487522][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-09-21T21:22:18.2200231Z][SCHEDULER][INFO]Scheduling job, id='HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38_0' 
[2022-09-21T21:22:18.3557519Z][SCHEDULER][INFO]Scheduling job, id='HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38_1' 
[2022-09-21T21:22:18.4671947Z][SCHEDULER][INFO]Scheduling job, id='HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38_2' 
[2022-09-21T21:22:18.5934699Z][SCHEDULER][INFO]Scheduling job, id='HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38_3' 
[2022-09-21T21:22:18.555441][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38',
 'target': 'StandardD2CPUCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-21T21:22:16.846221Z',
 'endTimeUtc': '2022-09-21T21:42:33.619703Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '5cf4050b-9c55-4576-be63-404eae912e05',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9102937606215101',
  'best_child_run_id': 'HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38_17',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38_17'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'te

In [5]:
import joblib
# Get your best run and save the model from that run.

# DW: First, get the best run
best_run = hyperdrive_run.get_best_run_by_primary_metric() # The best run based on the primary metric

print(best_run.id) # Display the ID of the best run
print(best_run.get_metrics()) # Display the parameters and primary metric values of the best run

HD_3ffa95d0-2053-4b11-8f34-c2483ad19c38_17
{'Regularization Strength:': 0.9054340254976188, 'Max iterations:': 100, 'Accuracy': 0.9102937606215101}


In [6]:
# DW: Next, identify the file names for the best run
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2022-09-21-21',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [7]:
# DW: Finally, register the model of the best run to save it
hd_model = best_run.register_model(model_name = 'hd_model',
                                   model_path = 'outputs/model.joblib')

## AutoML run on the same dataset

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# DW: Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dsurl = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(dsurl)

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
# DW: Let's first take a look at the data.
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [11]:
# DW: And now see what the output results look like.
y.value_counts()

0    29258
1     3692
Name: y, dtype: int64

So we have that many numerical field are present, and we want to use them to predict a 0 or 1 output. As such, this is a classification problem, just as was done with the logistic regression within the `train.py` script.

As the AutoML config will use just one dataset, `training_data`, we need to merge in the target column, as the cleaning process from the Udacity provided code has split the data into `x` with the various fields, and `y` with the target classification (0 or 1) that we want to predict. This target field can then be referred to as the `label_column_name` of the `training_data`.

Note that as per the below link, we do not need to split the data into training and testing sets as the AutoML will do this automatically, using a method depending on the number of rows of data.

https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-cross-validation-data-splits

In [12]:
# DW: Create a new column in dataframe x to store the target y values.
x['y'] = y
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


AutoML can only use a Pandas dataframe as the dataset if it is running locally. If it is running on a compute target, such as the compute cluster specified in this Jupyter Notebook, then the dataset must be coverted to a tabular dataset.  
Ref: https://knowledge.udacity.com/questions/469948

We can do this using `TabularDatasetFactory.register_pandas_dataframe` as detailed in the below link:  
https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.data.dataset_factory.tabulardatasetfactory?view=azure-ml-py

Or we can export the data to CSV and then upload it to a datastore in the workspace.

In [13]:
# DW: Save the Pandas dataframe as a CSV file within the training folder
x.to_csv('training/bankmarketing-train.csv', index=False)

In [14]:
# DW: List the datastores in the workspace
for datastore in ws.datastores:
    print(datastore)

print()
print('Default datastore: ',ws.get_default_datastore().name)

datastore
workspaceblobstore
workspaceworkingdirectory
workspaceartifactstore
workspacefilestore

Default datastore:  workspaceblobstore


In [15]:
# DW: Store the default datastore in a variable
datastore = ws.get_default_datastore()

In [16]:
# DW: Upload the training data to the datastore
datastore.upload(src_dir='./training', target_path='bankmarketing',
                 overwrite=True, show_progress=True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./training/bankmarketing-train.csv
Uploaded ./training/bankmarketing-train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_e8e2a93251e54141b5344a5730d737a9

In [17]:
# DW: Create our dataset object as a tabular data set for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(
    path=datastore.path('bankmarketing/bankmarketing-train.csv'))

In [18]:
from azureml.train.automl import AutoMLConfig

# DW: Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=cpu_cluster)

In [19]:
# DW: Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on StandardD2CPUCluster with default configuration
Running on remote compute: StandardD2CPUCluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e43e27f3-ae84-4faf-91df-aa84a77d48cb,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_e43e27f3-ae84-4faf-91df-aa84a77d48cb,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_e43e27f3-ae84-4faf-91df-aa84a77d48cb',
 'target': 'StandardD2CPUCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-21T22:09:32.17252Z',
 'endTimeUtc': '2022-09-21T22:50:31.005337Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'StandardD2CPUCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"48a74bb7-9950-4cc1-9caa-5d50f995cc55","resource_group":"aml-quickstarts-208238","workspace_name":"quick-starts-ws-208238","region":"southcentralus","compute_target":"StandardD2CPUCluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline

In [32]:
# Retrieve and save your best automl model.

# DW: First, get the best AutoML run ID and the model details
best_automl_run, best_automl_model = automl_run.get_output()

# Display these details
print(best_automl_run)
print()
print(best_automl_model)
print()
print(best_automl_run.get_metrics(name='accuracy'))

Run(Experiment: udacity-project,
Id: AutoML_e43e27f3-ae84-4faf-91df-aa84a77d48cb_32,
Type: azureml.scriptrun,
Status: Completed)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict=

In [33]:
# DW: Next, identify the file names for the best AutoML run
best_automl_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/63d315b1/classes.interpret.json',
 'explanation/63d315b1/expected_values.interpret.json',
 'explanation/63d315b1/features.interpret.json',
 'explanation/63d315b1/global_names/0.interpret.json',
 'explanation/63d315b1/global_rank/0.interpret.json',
 'explanation/63d315b1/global_values/0.interpret.json',
 'explanation/63d315b1/local_importance_values_sparse.interpret.json',
 'explanation/63d315b1/per_class_names/0.interpret.json',
 'explanation/63d315b1/per_class_rank/0.interpret.json',
 'explanation/63d315b1/per_class_values/0.interpret.json',
 'explanation/63d315b1/rich_metadata.interpret.json',
 'explanation/63d315b1/true_ys_viz.interpret.json',
 'explanation/63d315b1/visualization_dict.interpret.json',
 'explanation/63d315b1/ys_pred_proba_viz.interpret.json',
 'explanation/63d315b1/ys_pred_viz.interpret.json',
 'explanation/f7d12d6b/classes.interpret.json',
 'explanation/f7d12d6b/eval_data_viz.interpret.json',

In [35]:
# DW: Finally, register the model of the best AutoML run to save it
automl_model = best_automl_run.register_model(model_name = 'automl_model',
                                              model_path = 'outputs/model.pkl')

The two registered models, named `hd_model` and `automl_model`, can be viewed within the Azure ML Studio under the Models tab.

## Tidy up resources

In [38]:
# End the experiment logging that was started in the first code cell
run.complete()

# Finally, delete the CPU cluster
cpu_cluster.delete()